In [1]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [2]:
dict_table = pickle.load(open('dict_table','rb'))
df_historical_data = pd.read_csv('clean_fifa_champions_league_historical_data.csv')
df_fixture = pd.read_csv('clean_fifa_champions_league_fixture_data.csv')

In [3]:
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,Manchester City,Napoli,2011-2012,1,1,2
1,Villarreal,Bayern Munich,2011-2012,0,2,2
2,Bayern Munich,Manchester City,2011-2012,2,0,2
3,Napoli,Villarreal,2011-2012,2,0,2
4,Napoli,Bayern Munich,2011-2012,1,1,2
...,...,...,...,...,...,...
1243,Porto,Antwerp,2023-2024,2,0,2
1244,Shakhtar Donetsk,Antwerp,2023-2024,1,0,1
1245,Barcelona,Porto,2023-2024,2,1,3
1246,Porto,Shakhtar Donetsk,2023-2024,5,3,8


In [4]:
df_home = df_historical_data[['HomeTeam','HomeGoals','AwayGoals']]
df_away = df_historical_data[['AwayTeam','HomeGoals','AwayGoals']]

In [5]:
df_home = df_home.rename(columns={'HomeTeam': 'Team', 'HomeGoals':'GoalsScored','AwayGoals':'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam': 'Team', 'HomeGoals':'GoalsConceded','AwayGoals':'GoalsScored'})

In [6]:
df_team_strength = pd.concat([df_home,df_away],ignore_index = True).groupby('Team').mean()

In [7]:
df_team_strength

,GoalsScored,GoalsConceded
Team,,
1899 Hoffenheim,1.833333,2.333333
AEK Athens,0.333333,2.166667
APOEL,0.500000,1.944444
Ajax,1.629630,1.462963
Anderlecht,0.750000,2.208333
...,...,...
Viktoria Plzeň,0.916667,2.833333
Villarreal,1.166667,1.916667
Young Boys,1.000000,2.055556


In [8]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

In [9]:
predict_points('Borussia Dortmund','Barcelona')
predict_points('Bayern Munich','Barcelona')
predict_points('Barcelona','Zenit Saint Petersburg')

(2.4582894141525, 0.41167260903793945)

In [10]:
dict_table['Team'].values

array(['Arsenal', 'Aston Villa', 'Atalanta', 'Atlético Madrid',
       'Barcelona', 'Bayer Leverkusen', 'Bayern Munich', 'Benfica',
       'Bologna', 'Borussia Dortmund', 'Brest', 'Celtic', 'Club Brugge',
       'Dinamo Zagreb', 'Feyenoord', 'Girona', 'Inter Milan', 'Juventus',
       'Lille', 'Liverpool', 'Manchester City', 'Milan', 'Monaco',
       'Paris Saint-Germain', 'PSV Eindhoven', 'RB Leipzig',
       'Real Madrid', 'Red Bull Salzburg', 'Red Star Belgrade',
       'Shakhtar Donetsk', 'Slovan Bratislava', 'Sparta Prague',
       'Sporting CP', 'Sturm Graz', 'VfB Stuttgart', 'Young Boys'],
      dtype=object)

In [11]:
df_fixture

,home,score,away,year
0,Young Boys,v,Aston Villa,2024-2025
1,Juventus,v,PSV Eindhoven,2024-2025
2,Milan,v,Liverpool,2024-2025
3,Bayern Munich,v,Dinamo Zagreb,2024-2025
4,Real Madrid,v,VfB Stuttgart,2024-2025
...,...,...,...,...
139,PSV Eindhoven,v,Liverpool,2024-2025
140,Sturm Graz,v,RB Leipzig,2024-2025
141,Sporting CP,v,Bologna,2024-2025
142,Brest,v,Real Madrid,2024-2025


In [12]:
dict_table

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Arsenal,0,0,0,0,0,0,0,0
1,2,Aston Villa,0,0,0,0,0,0,0,0
2,3,Atalanta,0,0,0,0,0,0,0,0
3,4,Atlético Madrid,0,0,0,0,0,0,0,0
4,5,Barcelona,0,0,0,0,0,0,0,0
5,6,Bayer Leverkusen,0,0,0,0,0,0,0,0
6,7,Bayern Munich,0,0,0,0,0,0,0,0
7,8,Benfica,0,0,0,0,0,0,0,0
8,9,Bologna,0,0,0,0,0,0,0,0
9,10,Borussia Dortmund,0,0,0,0,0,0,0,0


In [13]:
# Получаем список всех команд
teams = dict_table['Team'].values

# Фильтруем df_fixture для матчей, в которых участвуют команды
df_fixture_filtered = df_fixture[(df_fixture['home'].isin(teams)) & (df_fixture['away'].isin(teams))]

# Проходим по каждому матчу
for index, row in df_fixture_filtered.iterrows():
    home, away = row['home'], row['away']
    points_home, points_away = predict_points(home, away)

    # Обновляем очки для команд
    dict_table.loc[dict_table['Team'] == home, 'Pts'] += points_home
    dict_table.loc[dict_table['Team'] == away, 'Pts'] += points_away

# Сортируем таблицу по очкам и обновляем её
dict_table = dict_table.sort_values('Pts', ascending=False).reset_index(drop=True)
dict_table = dict_table[['Team', 'Pts']]
dict_table = dict_table.round(0)

# Пример вывода обновленной таблицы
print("Updated Table:")
print(dict_table)

C:\Users\zveru\AppData\Local\Temp\ipykernel_11020\2847085017.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.39306174]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table.loc[dict_table['Team'] == home, 'Pts'] += points_home


Updated Table:
                   Team   Pts
0             Barcelona  16.0
1               Arsenal  15.0
2         Bayern Munich  15.0
3           Real Madrid  14.0
4             Liverpool  13.0
5              Juventus  13.0
6       Manchester City  12.0
7           Inter Milan  12.0
8     Borussia Dortmund  12.0
9       Atlético Madrid  12.0
10  Paris Saint-Germain  12.0
11                Milan  10.0
12          Sporting CP   8.0
13              Benfica   8.0
14           RB Leipzig   8.0
15             Atalanta   7.0
16     Bayer Leverkusen   7.0
17    Red Bull Salzburg   7.0
18               Monaco   6.0
19     Shakhtar Donetsk   6.0
20           Young Boys   6.0
21        PSV Eindhoven   5.0
22               Celtic   5.0
23            Feyenoord   5.0
24          Club Brugge   4.0
25                Lille   4.0
26    Red Star Belgrade   3.0
27        Dinamo Zagreb   2.0
28          Aston Villa   0.0
29               Girona   0.0
30                Brest   0.0
31    Slovan Bratislava  

In [14]:
for index, row in df_fixture_filtered.iterrows():
    home, away = row['home'], row['away']
    print(home)
    print(away)
    print('----------------------------------------------')

Young Boys
Aston Villa
----------------------------------------------
Juventus
PSV Eindhoven
----------------------------------------------
Milan
Liverpool
----------------------------------------------
Bayern Munich
Dinamo Zagreb
----------------------------------------------
Real Madrid
VfB Stuttgart
----------------------------------------------
Sporting CP
Lille
----------------------------------------------
Sparta Prague
Red Bull Salzburg
----------------------------------------------
Bologna
Shakhtar Donetsk
----------------------------------------------
Celtic
Slovan Bratislava
----------------------------------------------
Club Brugge
Borussia Dortmund
----------------------------------------------
Manchester City
Inter Milan
----------------------------------------------
Paris Saint-Germain
Girona
----------------------------------------------
Feyenoord
Bayer Leverkusen
----------------------------------------------
Red Star Belgrade
Benfica
-----------------------------------

In [18]:
predict_points('Lokomotiv Moscow','Zenit Saint Petersburg')

(0.704321218445139, 2.096286806898582)

In [20]:
predict_points('CSKA Moscow','Zenit Saint Petersburg')

(1.0951588335427394, 1.6971255689191498)

In [21]:
predict_points('Spartak Moscow','Zenit Saint Petersburg')

(0.9962535104545778, 1.8198141931952836)